# Maze Navigator

- This is a very important feature of TaskGen, as it helps with learning through tasks and dynamic adaptation of the Agent with changing environments
- `Shared Variables` help to keep track of persistent states for the agent
- Global Context can provide some of the `Shared Variables` as a global context to the Agent
- Here, we also utilise a Planner to help to come up with overall plan to guide actions

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI 'gpt-3.5-turbo' will be used as default

In [3]:
import time
import openai

def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    from openai import OpenAI
    import requests
    
    # Define your own LLM here
    client = OpenAI()
    
    def get_response():
        response = client.chat.completions.create(
            model='gpt-4o',
            temperature=0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return response.choices[0].message.content

    retries = 10  # Number of retries
    delay = 10  # Delay in seconds between retries

    for attempt in range(retries):
        try:
            start_time = time.time()
            response = get_response()
            elapsed_time = time.time() - start_time

            if elapsed_time > delay:
                raise requests.Timeout("Request took too long")

            return response
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                raise

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Maze Navigator
- We can use the global context to let agent know present state in a 2D grid, and obstacle positions that the agent has seen
- We use the `global_context` to configure more details about how we use the `shared_variables`
- Task: Given current position and end position, navigate to end position using actions: Up, Down, Left, Right

- The Maze Generation code is obtained from https://github.com/tanchongmin/Learning-Fast-and-Slow

In [5]:
def summarise_regions(tuples_list):
    if len(tuples_list) == 0:
        return "No Obstacles known"
    # Sort the list to ensure it is in order
    tuples_list.sort()

    # Initialize variables to track the current region
    regions = []
    current_start = tuples_list[0]
    current_end = tuples_list[0]

    for i in range(1, len(tuples_list)):
        current_tuple = tuples_list[i]
        # Check if the current tuple is a continuation of the current region
        if (current_tuple[0] == current_end[0] and current_tuple[1] == current_end[1] + 1) or (current_tuple[0] == current_end[0] + 1 and current_tuple[1] == current_end[1]):
            # Update the end of the current region
            current_end = current_tuple
        else:
            # Append the current region to the list
            regions.append((current_start, current_end))
            # Start a new region
            current_start = current_tuple
            current_end = current_tuple

    # Append the last region to the list
    regions.append((current_start, current_end))

    region_txt = ''
    for region in regions:
        region_txt += f"Obstacle from {region[0]} to {region[1]}\n"
    return region_txt

# Example usage
tuples_list = [
    (0, 20), (1, 20), (2, 20), (3, 20), (4, 20), (5, 20),
    (6, 20), (7, 20), (8, 20), (9, 20), (1, 3), (1, 4), (1, 5), (1, 6), (3, 4), (3, 5), (3, 6)
]

regions = summarise_regions(tuples_list)
print(regions)

Obstacle from (0, 20) to (0, 20)
Obstacle from (1, 3) to (1, 6)
Obstacle from (1, 20) to (2, 20)
Obstacle from (3, 4) to (3, 6)
Obstacle from (3, 20) to (9, 20)



In [6]:
# These are the utility functions
def generate_grid(size, episode = 0):
    '''Generates a grid with obstacles'''
    import numpy as np
    grid = [[' ' for _ in range(size)] for _ in range(size)]
    
    # Change episodes after 10 instances
    change_env = episode >= 10
    
    # define where the obstacles should be. 
    # This is following the setup in Learning, Fast and Slow (John and Motani, 2023)
    forbidden_squares = []
    if not change_env:
        forbidden_squares = [(x, size//2) for x in range(size) if x!=size//2]
    else:
        forbidden_squares = [(size//2, x) for x in range(size) if x!=size//2]
    
    for square in forbidden_squares:
        x, y = square
        grid[x][y] = 'O'
    
    # Generate the start state and end state
    np.random.seed(episode)
    # make the start state and goal state to be always across the border and not in forbidden squares
    if not change_env:
        start_pos = (np.random.randint(0, size), np.random.randint(0, size//2))
        goal_pos = (np.random.randint(0, size), np.random.randint(size//2 + 1, size))
        
    else:
        start_pos = (np.random.randint(0, size//2), np.random.randint(0, size))
        goal_pos = (np.random.randint(size//2 + 1, size), np.random.randint(0, size))
        
    grid[start_pos[0]][start_pos[1]] = 'S'  # Start
    grid[goal_pos[0]][goal_pos[1]] = 'E'  # Exit
        
    return start_pos, goal_pos, grid

def print_grid(grid):
    '''Prints the grid'''
    for row in grid:
        print(' '.join(row))

def update_obstacles(cur_pos, grid, grid_size, known_grid, known_obstacle_pos):
    '''Returns the new obstacles and updated known obstacle positions in the current grid given the cur_pos and grid_size'''
    row, col = cur_pos
    offset = [(x, y) for x in range(-1, 2) for y in range(-1, 2)]
    new_obs = []
    for row_offset, col_offset in offset:
        next_row, next_col = row+row_offset, col+col_offset
        # check if valid move
        if 0 <= next_row < grid_size and 0 <= next_col < grid_size:
            # Update known grid
            if grid[next_row][next_col] == 'O':
                known_grid[next_row][next_col] = 'O'
            elif known_grid[next_row][next_col] not in 'XE':
                known_grid[next_row][next_col] = ' '
                
            if grid[next_row][next_col] == 'O':
                new_obs.append((next_row, next_col))
                
            # adds in obstacle if observed
            if grid[next_row][next_col] == 'O' and (next_row, next_col) not in known_obstacle_pos:
                known_obstacle_pos.append((next_row, next_col))
            # remove obstacle that is not observed
            elif grid[next_row][next_col] != 'O' and (next_row, next_col) in known_obstacle_pos:
                known_obstacle_pos.remove((next_row, next_col))
    return new_obs, sorted(known_obstacle_pos)

In [7]:
def move(shared_variables, action: str, num_times: int):
    ''' Moves the agent according to the action for num_times and outputs outcome of action '''
    mapping = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1)}
    
    for i in range(num_times):
        # Retrieve from shared variables
        curpos = shared_variables["cur_pos"]
        row, col = curpos
        grid = shared_variables["grid"]
        grid_size = shared_variables["grid_size"]
        known_grid = shared_variables["known_grid"]
        known_obstacle_pos = shared_variables["known_obstacle_pos"]
        cells_visited = shared_variables["cells_visited"]

        # Do processing for the next action
        row_offset, col_offset = mapping[action]
        newpos = (row+row_offset, col+col_offset)
        
        # update obstacles
        new_obs, known_obstacle_pos = update_obstacles(newpos, grid, grid_size, known_grid, known_obstacle_pos)

        if newpos[0] < 0 or newpos[0] >= grid_size or newpos[1] < 0 or newpos[1] >= grid_size:
            cells_visited.append(curpos)
            my_agent.shared_variables["successful_last_action"] = False
            return f'''The current action of {action} from {curpos} will lead to going out of bounds of the maze at {newpos}
Obstacles sighted within 3x3 square: {new_obs}
Your final position: {shared_variables["cur_pos"]}.'''

        # if the move led to an obstacle, then return invalid move and update the known grid
        if grid[newpos[0]][newpos[1]] == 'O':
            cells_visited.append(curpos)
            known_grid[newpos[0]][newpos[1]] = 'O'
            my_agent.shared_variables["successful_last_action"] = False
            return f'''The current action of {action} from {curpos} led to an obstacle at {newpos}.
Obstacles sighted within 3x3 square: {new_obs}
Your final position: {shared_variables["cur_pos"]}'''

        # shift the current agent position
        grid[row][col] = ' '
        grid[row+row_offset][col+col_offset] = 'S'

        # update the known grid
        known_grid[row][col] = ' '
        known_grid[row+row_offset][col+col_offset] = 'S'  

        # Store back into shared variables
        shared_variables["cur_pos"] = newpos
        cells_visited.append(newpos)

    my_agent.shared_variables["successful_last_action"] = True
    return f'''Action successful. 
Obstacles sighted within 3x3 square: {new_obs}
Your final position: {newpos}'''

In [8]:
from collections import deque

def min_path_between_S_and_E(grid):
    rows, cols = len(grid), len(grid[0])
    start = None
    end = None
    
    # Find the start (S) and end (E) positions
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start = (r, c)
            elif grid[r][c] == 'E':
                end = (r, c)
    
    if not start or not end:
        return None
    
    # BFS setup
    queue = deque([(start[0], start[1], 0)])  # (row, col, distance)
    visited = set()
    visited.add(start)
    
    # Directions for moving in the grid (up, down, left, right)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    while queue:
        r, c, dist = queue.popleft()
        
        # Check if we have reached the end
        if (r, c) == end:
            return dist
        
        # Explore neighbors
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            
            if 0 <= nr < rows and 0 <= nc < cols and (nr, nc) not in visited:
                if grid[nr][nc] in (' ', '?', 'E'):  # Can move to passable cell or end cell
                    queue.append((nr, nc, dist + 1))
                    visited.add((nr, nc))
    
    return None  # If no path found

# Example grid
grid = [
    ['S', ' ', ' ', 'O', ' '],
    ['O', 'O', ' ', 'O', ' '],
    [' ', ' ', ' ', ' ', ' '],
    [' ', 'O', 'O', 'O', 'O'],
    [' ', ' ', ' ', ' ', 'E']
]

print(min_path_between_S_and_E(grid))

12


In [9]:
# Assign your agent
grid_size = 40
my_agent = Agent('Maze Navigator', 
                     f'''You are at Current Position, and are to move to the Exit Position of the maze.
A planner will provide you the subtask to get closer to Exit Position.
You are in a 2D gridworld with coordinates (row, col) according to numpy array convention
Top left of maze is (0, 0), bottom right is {(grid_size, grid_size)}
From current position (row, col), this is what you end up with after doing actions:
{{'Up': (row - 1, col), 'Down': (row + 1, col), 'Left': (row, col - 1), 'Right': (row, col + 1)}}
At each step, you must output an action - Up, Down, Left or Right
You can perform an action multiple times to save time''', 
                     shared_variables = {
                        "cur_pos": None,
                        "exit_pos": None,
                        "target_pos": None,
                        "known_grid": None,
                        "known_obstacle_pos": [],
                        "cells_visited": [],
                        "successful_last_action": False,
                        "grid_size": grid_size,
                        "grid": None}, 
                     max_subtasks = 1,
                     global_context = '''Current Pos: ```<cur_pos>```
Exit Pos: ```<exit_pos>```
''',
                     default_to_llm = False,
                     llm = llm).assign_functions([move])

In [10]:
def run_maze_env(my_agent, grid_size = 10, episode = 0):
    import random
    from IPython.display import clear_output
    
    # Generate a starting grid
    start_pos, exit_pos, grid = generate_grid(grid_size, episode)
    min_moves = min_path_between_S_and_E(grid)
    # reset cells visited
    my_agent.shared_variables["cells_visited"] = []
        
    # start with everything unknown
    known_grid = [['?' for _ in range(grid_size)] for _ in range(grid_size)]
    # add in already known obstacle positions
    for obs in my_agent.shared_variables["known_obstacle_pos"]:
        row, col = obs
        known_grid[row][col] = 'O'
    # add in Start and Exit positions
    known_grid[start_pos[0]][start_pos[1]] = 'S'
    known_grid[exit_pos[0]][exit_pos[1]] = 'E'
    
    # start with target pos at the Agent, so we can get planner to figure out where to head towards
    target_pos = start_pos

    # Initialise starting variables
    my_agent.shared_variables["cur_pos"] = start_pos
    my_agent.shared_variables["exit_pos"] = exit_pos
    my_agent.shared_variables["target_pos"] = target_pos
    my_agent.shared_variables["known_grid"] = known_grid
    my_agent.shared_variables["grid_size"] = grid_size
    my_agent.shared_variables["grid"] = grid
    
    # initialise empty plan
    plan = []
    my_agent.reset()
    completed = False
    
    for _ in range(100):
        # Planner only called if last action was not successful or no plan left
        if not my_agent.shared_variables["successful_last_action"] or len(plan) == 0:
            res = strict_json(f'''You are at Current Position, and are to move to the Exit Position of the maze.
You are in a 2D gridworld with coordinates (row, col) according to numpy array convention
Top left of maze is (0, 0), bottom right is {(grid_size, grid_size)}
From current position (row, col), this is what you end up with after doing actions:
{{'Up': (row - 1, col), 'Down': (row + 1, col), 'Left': (row, col - 1), 'Right': (row, col + 1)}}
You should perform an action multiple times whenever possible at each step of the Plan
For instance, ["Move down 1 time from (0, 0) to (0, 1)", "Move down 1 time from (0, 1) to (0, 2)"] 
can be rephrased as ["Move down 2 times from (0, 0) to (0, 2)"]
Your Plan cannot pass through any Obstacle Positions
You cannot take repeated actions through an obstacle. 
For instance, if there are Obstacles from (0, 1) to (5, 1), you cannot pass Move right 2 times from (0, 0) to (0, 2) as there is an Obstacle at (0, 1)

Example Start Position: (2, 0)
Example Exit Position: (2, 4)
Example Obstacle Positions: ["Obstacle from (0, 1) to (5, 1)"]
Example Obstacle Position Layout: There is a wall of obstacles from (0, 1) to (5, 1)
Example Thoughts: I need to get from (2, 0) to (0, 4)
There are obstacles in the way. Since (2, 1) to (5, 1) has obstacles, I am only able to go past the wall via (6, 1)
Example Plan: ["Move down 4 times from (2, 0) to (6, 0)", 
"Move right 4 times from (6, 0) to (6, 4)",
"Move up 4 times from (6, 4) to (2, 2)"]

Example Start Position: (0, 2)
Example Exit Position: (4, 2)
Example Obstacle Positions: ["Obstacle from (1, 0) to (1, 5)"]
Example Obstacle Position Layout: There is a wall of obstacles from (1, 0) to (1, 5)
Example Thoughts: I need to get from (0, 2) to (4, 0)
There are obstacles in the way. Since (1, 0) to (1, 5) has obstacles, I am only able to go past the wall via (1, 6)
Example Plan: ["Move right 4 times from (0, 2) to (0, 6)", 
"Move down 4 times from (0, 6) to (4, 6)",
"Move left 4 times from (4, 6) to (4, 2)"]

Example Start Position: (3, 2)
Example Exit Position: (4, 5)
Example Obstacle Positions: ["Obstacle from (0, 3) to (2, 3)", "Obstacle from (4, 3) to (5, 3)"]
Example Obstacle Position Layout: There is a wall of obstacles from (0, 3) to (5, 3), with a gap at (3, 3)
Example Thoughts: I need to get from (3, 2) to (4, 5)
I should go through the gap in the wall at (3, 3) to get from (3, 2) to (3, 5), then move to (4, 5)
Example Plan: ["Move right 3 times from (3, 2) to (3, 5)", "Move down 1 time from (3, 5) to (4, 5)"]

Example Start Position: (2, 3)
Example Exit Position: (5, 4)
Example Obstacle Positions: ["Obstacle from (3, 0) to (3, 2)", "Obstacle from (3, 4) to (3, 5)"]
Example Obstacle Position Layout: There is a wall of obstacles from (3, 0) to (3, 5), with a gap at (3, 3)
Example Thoughts: I need to get from (2, 3) to (5, 4)
I should go through the gap in the wall at (3, 3) to get from (2, 3) to (5, 3), then move to (4, 5)
Example Plan: ["Move down 3 times from (2, 3) to (5, 3)", "Move right 1 time from (5, 3) to (5, 4)"]
''',
            f'''Previous Subtasks: {my_agent.subtasks_completed}
Current Position: {my_agent.shared_variables["cur_pos"]}, 
Exit Position: {my_agent.shared_variables["exit_pos"]}
Obstacle Positions: {summarise_regions(my_agent.shared_variables["known_obstacle_pos"])}''',
            output_format = {"Obstacle Position Layout": "Describe where Obstacles are, highlight walls of Obstacles and note for any gaps",
                             "Thoughts": "What to do to avoid obstacles",
                             "Plan": "List of actions to do to get to exit and avoid obstacles, type: list"},
                              llm = llm)

            print(res)
            plan = res['Plan']
            
        # Executor
        cur_plan = plan.pop(0)
        my_agent.reset()
        my_agent.run(cur_plan)
        
        if my_agent.shared_variables["cur_pos"] == my_agent.shared_variables["exit_pos"]: 
            completed = True
            break
    
    return completed, len(my_agent.shared_variables["cells_visited"]), min_moves

# Orientation 1: Vertical

In [11]:
run_maze_env(my_agent, 40, 0)

{'Obstacle Position Layout': 'No obstacles present in the maze.', 'Thoughts': 'Since there are no obstacles, I can move directly towards the exit position.', 'Plan': ['Move down 3 times from (0, 3) to (3, 3)', 'Move right 25 times from (3, 3) to (3, 28)']}
Observation: The Assigned Task is to move down 3 times from (0, 3) to (3, 3). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 3 times. This will take me from (0, 3) to (3, 3).
Subtask identified: Move down 3 times from (0, 3) to (3, 3)
Calling function move with parameters {'action': 'Down', 'num_times': 3}
> {'output_1': 'Action successful. \nObstacles sighted within 3x3 square: []\nYour final position: (3, 3)'}

Observation: The Assigned Task is to move right 25 times from (3, 3) to (3, 28). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move right 25 times. Since I can perform an action multiple times, I can execute the move right action 25 

(True, 210, 62)

In [12]:
run_maze_env(my_agent, 40, 1)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and a single obstacle at (21, 20).', 'Thoughts': 'I need to get from (37, 11) to (12, 29). There is a wall of obstacles from (0, 20) to (19, 20) and a single obstacle at (21, 20). I should navigate around the wall by moving up to row 20, then move right past the wall, and finally move up to the exit position.', 'Plan': ['Move up 17 times from (37, 11) to (20, 11)', 'Move right 9 times from (20, 11) to (20, 20)', 'Move up 8 times from (20, 20) to (12, 20)', 'Move right 9 times from (12, 20) to (12, 29)']}
Observation: The Assigned Task is to move up 17 times from (37, 11) to (20, 11). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move up 17 times. Since no subtasks have been completed, I can start by moving up.
Subtask identified: Move up 17 times from (37, 11) to (20, 11).
Calling function move with parameters {'action': 'Up', 'num_times': 17}
> {'output_1': 'Ac

(True, 46, 43)

In [13]:
run_maze_env(my_agent, 40, 2)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall from (21, 20) to (22, 20). There is a gap at row 20.', 'Thoughts': 'I need to get from (15, 13) to (8, 32). To avoid the obstacles, I should move up to row 20, then move right past the gap, and finally move up to the exit position.', 'Plan': ['Move up 5 times from (15, 13) to (10, 13)', 'Move up 5 times from (10, 13) to (5, 13)', 'Move right 7 times from (5, 13) to (5, 20)', 'Move right 12 times from (5, 20) to (5, 32)', 'Move down 3 times from (5, 32) to (8, 32)']}
Observation: The Assigned Task is to move up 5 times from (15, 13) to (10, 13).
Thoughts: To complete the Assigned Task, I need to move up 5 times from the current position (15, 13) to reach (10, 13).
Subtask identified: Move up 5 times from (15, 13) to (10, 13).
Calling function move with parameters {'action': 'Up', 'num_times': 5}
> {'output_1': 'Action successful. \nObstacles sighted within 3x3 square: []\nYour final posi

(True, 58, 36)

In [14]:
run_maze_env(my_agent, 40, 3)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall from (21, 20) to (22, 20). There is a gap at (20, 20).', 'Thoughts': 'I need to get from (24, 3) to (8, 21). To avoid the obstacles, I should move up to row 20, then move right to column 21, and finally move up to the exit position.', 'Plan': ['Move up 4 times from (24, 3) to (20, 3)', 'Move right 17 times from (20, 3) to (20, 20)', 'Move right 1 time from (20, 20) to (20, 21)', 'Move up 12 times from (20, 21) to (8, 21)']}
Observation: The Assigned Task is to move up 4 times from (24, 3) to (20, 3). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move up 4 times from the current position (24, 3) to reach (20, 3).
Subtask identified: Move up 4 times from (24, 3) to (20, 3).
Calling function move with parameters {'action': 'Up', 'num_times': 4}
> {'output_1': 'Action successful. \nObstacles sighted within 3x3 square: []\nYour final position: (20, 3

(True, 34, 34)

In [15]:
run_maze_env(my_agent, 40, 4)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall from (21, 20) to (22, 20). There is a gap at row 20.', 'Thoughts': 'I need to get from (5, 1) to (23, 29). To avoid the obstacles, I should move down to row 20, then move right past the gap, and finally move down to row 23 and right to column 29.', 'Plan': ['Move down 15 times from (5, 1) to (20, 1)', 'Move right 20 times from (20, 1) to (20, 21)', 'Move down 3 times from (20, 21) to (23, 21)', 'Move right 8 times from (23, 21) to (23, 29)']}
Observation: The Assigned Task is to move down 15 times from (5, 1) to (20, 1). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 15 times. This will take me from the current position (5, 1) to the target position (20, 1).
Subtask identified: Move down 15 times from (5, 1) to (20, 1).
Calling function move with parameters {'action': 'Down', 'num_times': 15}
> {'output_1': 'Action successful. \nObstacl

(True, 46, 46)

In [16]:
run_maze_env(my_agent, 40, 5)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall from (21, 20) to (24, 20). There is a gap between (20, 20) and (21, 20).', 'Thoughts': 'I need to get from (35, 14) to (38, 37). I should avoid the obstacles by moving around them. Since the obstacles are at column 20, I can move right until I pass column 20, then move down and right to reach the exit.', 'Plan': ['Move right 7 times from (35, 14) to (35, 21)', 'Move down 3 times from (35, 21) to (38, 21)', 'Move right 16 times from (38, 21) to (38, 37)']}
Observation: The Assigned Task is to move right 7 times from (35, 14) to (35, 21). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move right 7 times. Since no subtasks have been completed, I should start by moving right.
Subtask identified: Move right 7 times from (35, 14) to (35, 21).
Calling function move with parameters {'action': 'Right', 'num_times': 7}
> {'output_1': 'The current action of

(True, 182, 56)

In [17]:
run_maze_env(my_agent, 40, 6)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall from (21, 20) to (39, 20), with a gap at (20, 20).', 'Thoughts': 'I need to get from (10, 9) to (35, 31). There are obstacles in the way. Since there are walls of obstacles from (0, 20) to (19, 20) and from (21, 20) to (39, 20), I am only able to go past the wall via (20, 20).', 'Plan': ['Move right 11 times from (10, 9) to (10, 20)', 'Move down 10 times from (10, 20) to (20, 20)', 'Move down 15 times from (20, 20) to (35, 20)', 'Move right 11 times from (35, 20) to (35, 31)']}
Observation: The Assigned Task is to move right 11 times from the current position (10, 9) to reach (10, 20). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move right 11 times. This will take me from (10, 9) to (10, 20).
Subtask identified: Move right 11 times from (10, 9) to (10, 20).
Calling function move with parameters {'action': 'Right', 'num_times': 11}
> {'output_1

(True, 49, 47)

In [18]:
run_maze_env(my_agent, 40, 7)

{'Obstacle Position Layout': 'There is a continuous wall of obstacles from (0, 20) to (19, 20) and another continuous wall from (21, 20) to (39, 20). There is a gap at row 20.', 'Thoughts': 'I need to get from (4, 3) to (19, 28). To avoid the obstacles, I should move right until I reach column 19, then move down to row 20 to pass through the gap, and finally move right to column 28 and down to row 19.', 'Plan': ['Move right 16 times from (4, 3) to (4, 19)', 'Move down 16 times from (4, 19) to (20, 19)', 'Move right 9 times from (20, 19) to (20, 28)', 'Move up 1 time from (20, 28) to (19, 28)']}
Observation: The Assigned Task is to move right 16 times from (4, 3) to (4, 19). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move right 16 times. Since no subtasks have been completed, I can start by moving right.
Subtask identified: Move right 16 times from (4, 3) to (4, 19)
Calling function move with parameters {'action': 'Right', 'num_times': 16}
> 

(True, 42, 42)

In [19]:
run_maze_env(my_agent, 40, 8)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall from (21, 20) to (39, 20), with a gap at (20, 20).', 'Thoughts': 'I need to get from (3, 17) to (5, 29). There are obstacles in the way. Since there is a gap at (20, 20), I should navigate around the obstacles by moving down to row 20, then move right past the gap, and finally move up to the exit position.', 'Plan': ['Move down 17 times from (3, 17) to (20, 17)', 'Move right 3 times from (20, 17) to (20, 20)', 'Move right 9 times from (20, 20) to (20, 29)', 'Move up 15 times from (20, 29) to (5, 29)']}
Observation: The Assigned Task is to move down 17 times from (3, 17) to (20, 17). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 17 times from the current position (3, 17).
Subtask identified: Move down 17 times from (3, 17) to (20, 17).
Calling function move with parameters {'action': 'Down', 'num_times': 17}
> {'output_1': 'Action succe

(True, 44, 44)

In [20]:
run_maze_env(my_agent, 40, 9)

Attempt 1 failed: Request took too long
{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall of obstacles from (21, 20) to (39, 20). There is a gap at row 20.', 'Thoughts': 'I need to get from (28, 1) to (22, 29). To avoid the obstacles, I should move up to row 20 where there is a gap, then move right past the obstacles, and finally move up to the exit position.', 'Plan': ['Move up 8 times from (28, 1) to (20, 1)', 'Move right 19 times from (20, 1) to (20, 20)', 'Move right 9 times from (20, 20) to (20, 29)', 'Move up 2 times from (20, 29) to (22, 29)']}
Observation: The Assigned Task is to move up 8 times from (28, 1) to (20, 1). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move up 8 times. This will take me from (28, 1) to (20, 1).
Subtask identified: Move up 8 times from (28, 1) to (20, 1).
Calling function move with parameters {'action': 'Up', 'num_times': 8}
> {'output_1': 'Action successful

(True, 38, 38)

# Orientation 2: Horizontal

In [21]:
run_maze_env(my_agent, 40, 10)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (19, 20) and another wall of obstacles from (21, 20) to (39, 20), with a gap at row 20.', 'Thoughts': 'I need to get from (9, 36) to (36, 0). There are obstacles in the way. Since there are walls of obstacles from (0, 20) to (19, 20) and from (21, 20) to (39, 20), I can only pass through the gap at row 20.', 'Plan': ['Move down 11 times from (9, 36) to (20, 36)', 'Move left 16 times from (20, 36) to (20, 20)', 'Move down 16 times from (20, 20) to (36, 20)', 'Move left 20 times from (36, 20) to (36, 0)']}
Observation: The Assigned Task is to move down 11 times from the current position (9, 36) to (20, 36). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 11 times. This will take me from (9, 36) to (20, 36).
Subtask identified: Move down 11 times from (9, 36) to (20, 36).
Calling function move with parameters {'action': 'Down', 'num_times': 11}
> {'output_1': 'The c

(True, 68, 63)

In [22]:
run_maze_env(my_agent, 40, 11)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (17, 20). There are isolated obstacles at (20, 18) and (20, 19). There is another wall of obstacles from (20, 21) to (20, 38). Finally, there is a wall of obstacles from (24, 20) to (39, 20).', 'Thoughts': 'I need to get from (16, 27) to (38, 13). There are multiple walls of obstacles blocking direct paths. I should navigate around these walls by moving down to avoid the first wall, then move right to avoid the second wall, and finally move down again to avoid the third wall.', 'Plan': ['Move down 4 times from (16, 27) to (20, 27)', 'Move right 12 times from (20, 27) to (20, 39)', 'Move down 4 times from (20, 39) to (24, 39)', 'Move left 19 times from (24, 39) to (24, 20)', 'Move down 14 times from (24, 20) to (38, 20)', 'Move left 7 times from (38, 20) to (38, 13)']}
Observation: The Assigned Task is to move down 4 times from (16, 27) to (20, 27). No subtasks have been completed yet.
Thoughts: To complete the A

(True, 62, 36)

In [23]:
run_maze_env(my_agent, 40, 12)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (17, 20). There is a small wall of obstacles at (20, 18) to (20, 19). There is another wall of obstacles from (20, 21) to (20, 39). Finally, there is a wall of obstacles from (26, 20) to (39, 20).', 'Thoughts': 'I need to get from (11, 27) to (27, 2). The wall of obstacles from (0, 20) to (17, 20) blocks a direct path downwards. I need to navigate around this wall. I can move down to row 18, then move left to column 19, then move down to row 26, and finally move left to column 2.', 'Plan': ['Move down 7 times from (11, 27) to (18, 27)', 'Move left 8 times from (18, 27) to (18, 19)', 'Move down 8 times from (18, 19) to (26, 19)', 'Move left 17 times from (26, 19) to (26, 2)', 'Move down 1 time from (26, 2) to (27, 2)']}
Observation: The Assigned Task is to move down 7 times from (11, 27) to (18, 27). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 7 times. This wi

(True, 52, 41)

In [24]:
run_maze_env(my_agent, 40, 13)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (16, 20). There is a wall of obstacles from (20, 16) to (20, 19) and another wall from (20, 21) to (20, 39), with a gap at (20, 20). There is a wall of obstacles from (28, 20) to (39, 20).', 'Thoughts': 'I need to get from (18, 10) to (37, 38). There are multiple walls of obstacles. I should navigate through the gap at (20, 20) and avoid the walls by moving around them.', 'Plan': ['Move down 2 times from (18, 10) to (20, 10)', 'Move right 10 times from (20, 10) to (20, 20)', 'Move down 8 times from (20, 20) to (28, 20)', 'Move right 18 times from (28, 20) to (28, 38)', 'Move down 9 times from (28, 38) to (37, 38)']}
Observation: The Assigned Task is to move down 2 times from (18, 10) to (20, 10). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 2 times from the current position (18, 10) to reach (20, 10).
Subtask identified: Move down 2 times from (18, 10) to (20,

(True, 48, 47)

In [25]:
run_maze_env(my_agent, 40, 14)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (16, 20). There is another wall of obstacles from (20, 9) to (20, 19) and from (20, 21) to (20, 39), with a gap at (20, 20). Additionally, there is a wall of obstacles from (30, 20) to (39, 20).', 'Thoughts': 'I need to get from (11, 24) to (33, 22). To avoid the obstacles, I should move down to pass the first wall at (16, 20), then navigate through the gap at (20, 20), and finally move down again to pass the last wall at (30, 20).', 'Plan': ['Move down 5 times from (11, 24) to (16, 24)', 'Move left 4 times from (16, 24) to (16, 20)', 'Move down 4 times from (16, 20) to (20, 20)', 'Move down 10 times from (20, 20) to (30, 20)', 'Move down 3 times from (30, 20) to (33, 20)', 'Move right 2 times from (33, 20) to (33, 22)']}
Observation: The Assigned Task is to move down 5 times from (11, 24) to (16, 24). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 5 times from 

(True, 28, 28)

In [26]:
run_maze_env(my_agent, 40, 15)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (14, 20). There is another wall of obstacles from (20, 9) to (20, 19) and from (20, 21) to (20, 39), with a gap at (20, 20). Additionally, there is a wall of obstacles from (35, 20) to (39, 20).', 'Thoughts': 'I need to get from (8, 12) to (26, 0). To avoid the obstacles, I should move down to row 20, then move right to column 20 through the gap at (20, 20), and finally move down to row 26 and left to column 0.', 'Plan': ['Move down 12 times from (8, 12) to (20, 12)', 'Move right 8 times from (20, 12) to (20, 20)', 'Move down 6 times from (20, 20) to (26, 20)', 'Move left 20 times from (26, 20) to (26, 0)']}
Observation: The Assigned Task is to move down 12 times from (8, 12) to (20, 12). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 12 times. This will take me from the current position (8, 12) to the target position (20, 12).
Subtask identified: Move down 12 t

(True, 75, 46)

In [27]:
run_maze_env(my_agent, 40, 16)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (14, 20). There is another wall of obstacles from (20, 0) to (20, 19) and from (20, 21) to (20, 39), with a gap at (20, 20). Additionally, there is a wall of obstacles from (35, 20) to (39, 20).', 'Thoughts': 'I need to get from (9, 5) to (22, 4). I should avoid the obstacles by moving down to row 19, then move right to column 4, and finally move down to row 22.', 'Plan': ['Move down 10 times from (9, 5) to (19, 5)', 'Move left 1 time from (19, 5) to (19, 4)', 'Move down 3 times from (19, 4) to (22, 4)']}
Observation: The Assigned Task is to move down 10 times from (9, 5) to (19, 5). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 10 times from the current position (9, 5). This will bring me to the position (19, 5).
Subtask identified: Move down 10 times from (9, 5) to (19, 5).
Calling function move with parameters {'action': 'Down', 'num_times': 10}
> {'output_1

(True, 47, 44)

In [28]:
run_maze_env(my_agent, 40, 17)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (14, 20). There is another wall of obstacles from (20, 0) to (20, 19) and from (20, 21) to (20, 39), with a gap at (20, 20). Additionally, there is a wall of obstacles from (35, 20) to (39, 20).', 'Thoughts': 'I need to get from (15, 15) to (27, 22). To avoid the obstacles, I should move down to (20, 15), then move right through the gap at (20, 20), and finally move down and right to reach the exit.', 'Plan': ['Move down 5 times from (15, 15) to (20, 15)', 'Move right 5 times from (20, 15) to (20, 20)', 'Move down 7 times from (20, 20) to (27, 20)', 'Move right 2 times from (27, 20) to (27, 22)']}
Observation: The Assigned Task is to move down 5 times from (15, 15) to (20, 15). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 5 times. This will take me from (15, 15) to (20, 15).
Subtask identified: Move down 5 times from (15, 15) to (20, 15).
Calling function move

(True, 21, 19)

In [29]:
run_maze_env(my_agent, 40, 18)

{'Obstacle Position Layout': 'There is a wall of obstacles from (0, 20) to (14, 20). There is another wall of obstacles from (20, 0) to (20, 19) and from (20, 21) to (20, 39), with a gap at (20, 20). Additionally, there is a wall of obstacles from (35, 20) to (39, 20).', 'Thoughts': 'I need to get from (10, 19) to (26, 2). To avoid the obstacles, I should move down to (15, 19), then move right to (15, 21) to bypass the first wall. Then, I should move down to (20, 21) and move left through the gap at (20, 20). After that, I can move down to (26, 20) and finally move left to (26, 2).', 'Plan': ['Move down 5 times from (10, 19) to (15, 19)', 'Move right 2 times from (15, 19) to (15, 21)', 'Move down 5 times from (15, 21) to (20, 21)', 'Move left 1 time from (20, 21) to (20, 20)', 'Move down 6 times from (20, 20) to (26, 20)', 'Move left 18 times from (26, 20) to (26, 2)']}
Observation: The Assigned Task is to move down 5 times from (10, 19) to (15, 19). No subtasks have been completed yet

(True, 57, 35)

In [30]:
run_maze_env(my_agent, 40, 19)

{'Obstacle Position Layout': 'There are multiple walls of obstacles: from (0, 20) to (9, 20), from (20, 0) to (20, 19), from (20, 21) to (20, 39), and from (37, 20) to (39, 20). There is a gap at (20, 20).', 'Thoughts': 'I need to get from (14, 34) to (29, 19). I should navigate around the obstacles by moving down to the gap at (20, 20), then move through the gap and continue to the exit.', 'Plan': ['Move down 6 times from (14, 34) to (20, 34)', 'Move left 14 times from (20, 34) to (20, 20)', 'Move down 9 times from (20, 20) to (29, 20)', 'Move left 1 time from (29, 20) to (29, 19)']}
Observation: The Assigned Task is to move down 6 times from (14, 34) to (20, 34). No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, I need to move down 6 times. This will take me from (14, 34) to (20, 34).
Subtask identified: Move down 6 times from (14, 34) to (20, 34).
Calling function move with parameters {'action': 'Down', 'num_times': 6}
> {'output_1': 'The current action o

(True, 34, 30)